# Fase 4: Busca de Parâmetros

Nesta fase iremos aprofundar a busca de hiperparamewtros, variabndo os parametros ca cada modelos, e compaerandos seus resultados

O primeiro passo que iremos fazer é importar as bibliotecas principais, e estabelecer o ambiente de execução.


In [1]:
# Bibliotecas Externas
import os
import warnings
import pickle
import itertools
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Ignorando Warnings
warnings.filterwarnings("ignore")

# Bibliotecas Locais
os.chdir("../..")
from lib import plots, runner

# Definido Variáveis globais
N_SPLITS = 5
RANDOM_STATE = 51

## 0. Importando os Dados

Nosso primero é coletar os dados que preparamos na etapa anterior, para verificar como esses dados se comportam

In [2]:
# Conjunto de treino
X_train = pd.read_csv('./data/processed/X_train.csv')
Y_train = pd.read_csv('./data/processed/Y_train.csv')

Y_train['class'] = Y_train['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de validação
X_val = pd.read_csv('./data/processed/X_val.csv')
Y_val = pd.read_csv('./data/processed/Y_val.csv')


Y_val['class'] = Y_val['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de teste
X_test = pd.read_csv('./data/processed/X_test.csv')
Y_test = pd.read_csv('./data/processed/Y_test.csv')

Y_test['class'] = Y_test['class'].apply(lambda val: 1 if val == 'UP' else 0)

## 1. Carregando os modelos para fazer o Stacking

In [ ]:
# Lista com os nomes dos modelos -> TALVEZ VARIAR AQUI!!!
model_files = {
    "decision_tree": "./models/decision_tree.pkl",
    "knn": "./models/knn.pkl",
    "lightgbm": "./models/lightgbm.pkl",
    # "mlp": "./models/mlp.pkl",
    "svm": "./models/svm.pkl",
    "xgboost": "./models/xgboost.pkl",
    "random_forest": "./models/random_forest.pkl"
}

# Criar um dicionário para armazenar os modelos carregados
models_l0 = []

# Carregar os modelos usando pickle
for name, file in model_files.items():
    with open(file, "rb") as f:
        models_l0.append((name, pickle.load(f)))

## 2. Algoritmos de Machine Learning para Classificação

### Stacking

In [5]:
model_l1 = LogisticRegression(
    random_state=RANDOM_STATE
)

In [6]:
stack = StackingClassifier(
    estimators=models_l0,
    final_estimator=model_l1,
    cv="prefit"
)

In [32]:
stack.fit(X_train, Y_train)

StackingClassifier(cv='prefit',
                   estimators=[('knn',
                                KNeighborsClassifier(metric='manhattan',
                                                     n_neighbors=7,
                                                     weights='distance')),
                               ('lightgbm',
                                LGBMClassifier(colsample_bytree=0.9,
                                               max_depth=20, n_estimators=250,
                                               n_jobs=-1, num_leaves=40,
                                               objective='binary',
                                               random_state=51, reg_alpha=0,
                                               reg_lambda=0.01,
                                               subsample=0.6)),
                               ('mlp',
                                MLPClassifier(activation='logistic',
                                              a...
                                              max_leaves=None,
                                              min_child_weight=3, missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=-1,
                                              num_parallel_tree=None, ...)),
                               ('random_forest',
                                RandomForestClassifier(bootstrap=False,
                                                       max_depth=50,
                                                       min_samples_leaf=2,
                                                       min_samples_split=5,
                                                       n_estimators=500,
                                                       n_jobs=-1,
                                                       random_state=51))],
                   final_estimator=LogisticRegression(random_state=51))

In [33]:
df_stack = runner.evaluate_model(stack, X_train, Y_train, X_test, Y_test)

In [34]:
df_stack

,Dataset,F1_Score,Recall,AUC,Accuracy,FPR,TPR
0,train,0.978212,0.976952,0.981128,0.981814,"[0.0, 0.014695529743048222, 1.0]","[0.0, 0.9769523108986147, 1.0]"
1,test,0.889152,0.881078,0.905820,0.910373,"[0.0, 0.06943883730318934, 1.0]","[0.0, 0.8810779144698301, 1.0]"
